In [28]:
import requests
import pandas as pd
import requests
import json
from tqdm import tqdm
from pathlib import Path
import re

In [31]:
def download_pdf(doi, dest_folder, content):
    doi_suffix = str(doi.split('/', 1)[1:]).strip("[']")
    doi_suffix = re.sub(r"""([()/\\*,"': ?;<>])""", '_._', doi_suffix)
    filename = dest_folder + '/' + str(doi_suffix)+'.pdf'
    with open(filename, mode = 'wb') as f:
        f.write(content)

In [23]:
with open('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/code/cpet_articles/gathering/full-text_download_code/springer/springer_config.json') as config_file:
    api_key = json.load(config_file)['api_key']

In [24]:
articles = pd.read_csv('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/unpaywall/unpaywall_info.csv')
springer_non_oa_articles = articles[(articles['is_oa'] == False) & \
    ((articles['publisher'] == 'Springer Science and Business Media LLC') | \
     (articles['publisher'] == 'Springer Nature') | \
     (articles['publisher'] == 'Springer International Publishing'))].drop_duplicates().reset_index(drop=True)

In [25]:
springer_non_oa_articles['publisher'].value_counts()

Springer Science and Business Media LLC    2276
Springer Nature                             205
Springer International Publishing            31
Name: publisher, dtype: int64

In [26]:
pdf_file_paths = list(Path('/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs').rglob('*/*.pdf'))
pdfs = [path.stem for path in pdf_file_paths]

In [29]:
re_doi_suffix = re.compile(r'(?<=\d/).*')
springer_non_oa_articles['doi_suffix'] = springer_non_oa_articles['doi'].apply(lambda x: re_doi_suffix.search(x).group())
full_texts_to_download = [x for x in springer_non_oa_articles['doi_suffix'].tolist() if x not in pdfs]
merge = pd.merge(pd.DataFrame({'doi_suffix': full_texts_to_download}), springer_non_oa_articles, how='inner', on='doi_suffix')
merge.shape

(296, 50)

In [41]:
springer_api_url = 'https://api.springernature.com/meta/v2/json?'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:100.0) Gecko/20100101 Firefox/100.0'}
folder = '/Users/antonhesse/Desktop/Anton/Education/UMN/Lab and Research/HSPL/CPET_scoping_review/data/cpet_articles/full_texts/pdfs/springer_non_oa'

In [42]:
log = []

In [43]:
for i, row in tqdm(merge.iterrows(), total=merge.shape[0]):
    doi = row['doi']
    out = {'doi': doi}
    params = {
    'api_key': api_key,
    'q': doi
    }

    try:
        response = requests.get(url = springer_api_url, params = params, headers=headers, allow_redirects=True)
        out.update({'query_status_code': response.status_code})
                    
        if response.status_code == 200:
            pdf_url = response.json()['records'][0]['url'][1]['value']
            pdf_response = requests.get(pdf_url, stream=True, allow_redirects=True)
            out.update({'pdf_status_code': pdf_response.status_code})
                    
            if pdf_response.status_code == 200:
                download_pdf(doi, dest_folder=folder, content=pdf_response.content)

    except Exception as e:
        print(f'{e} at index {i} for DOI {doi}')
        out.update({'error': e})
    
    log.append(out)


 11%|█         | 33/296 [02:03<14:02,  3.20s/it]

HTTPSConnectionPool(host='41110.springer.com', port=443): Max retries exceeded with url: /track/pdf/10.1186/s41110-020-00121-5.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fae39b4bac0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')) at index 32 for DOI 10.1007/s00421-009-1351-8


 12%|█▏        | 35/296 [02:07<11:38,  2.68s/it]

HTTPSConnectionPool(host='40200.springer.com', port=443): Max retries exceeded with url: /track/pdf/10.1007/s40200-021-00759-6.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fae39970eb0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')) at index 34 for DOI 10.1007/s00421-006-0294-6


 23%|██▎       | 67/296 [03:44<08:34,  2.25s/it]

...

 73%|███████▎  | 215/296 [13:18<03:10,  2.35s/it]

list index out of range at index 214 for DOI 10.1038/clpt.1983.96


 77%|███████▋  | 229/296 [14:12<04:54,  4.40s/it]

list index out of range at index 228 for DOI 10.1038/clpt.1980.107


100%|██████████| 296/296 [17:09<00:00,  3.48s/it]


In [ ]:
log_df = pd.DataFrame(log)